In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np

from azure_blob import download_blob_file
from utilities import string_to_dict

In [ ]:
# Download du .csv originel depuis le serveur de stockage

filename = "metrics.csv"
path = "../data/metrics/"

download_blob_file(file_name=filename , local_path=path)

In [ ]:
# Création du Dataframe à partir du .csv importé
# Filtration du Df via la colonne "created_at"
# Suppression de toutes les colonnes vides

df= pd.read_csv(path + filename).sort_values("created_at").dropna(axis=1, how="all",)
df.info()
df.head()

In [ ]:
# Modification du Dataframe pour ne garder que les colonnes qui nous intéresse

df = df[["id","status","created_at","events"]]
df.head()

In [ ]:
# Suppression de toutes les lignes sans données dans la colonne "events"
# Remise de l'index à zéro

df = df[df.events != "[]"].reset_index(drop=True)
df.head()

In [ ]:
## Modification du Dataframe pour ne garder que les colonnes qui nous intéresse
## Suppression de toutes les lignes sans données dans la colonne "events"
## Remise de l'index à zéro

df = df[["id","status","created_at","events"]][df.events != "[]"].reset_index(drop=True)
# df.head()

In [ ]:
# Création d'un .csv permettant de sauvegardé les modification faites

df.to_csv(path + "metrics_events.csv")

In [ ]:
def convert_str_to_list(series):
    return pd.DataFrame(series.apply(lambda x :json.loads(x)), columns=[series.name])

In [ ]:
df_events = convert_str_to_list(df.events)
df_events.head()

In [ ]:
events_keys = []
for i in range(0, len(df_events)) :
    if len(df_events.events.loc[i]) != 0 :
        event_keys = df_events.events.loc[i][0].keys()
        if event_keys not in events_keys :
            events_keys.append(event_keys)

In [ ]:
events_dict = {}
for key in event_keys :
    events_dict[key] = None

In [ ]:
df_events['Length'] = df_events.events.map(len)

In [ ]:
df_events = pd.DataFrame(df_events.events.to_list(), dtype=object)

df_events.info()
df_events.head()

In [ ]:
for col in df_events.columns :
    for i in range(0,len(df_events.iloc[:,col])):
        if isinstance(df_events.loc[i][col], type(None)) :
            df_events.loc[i][col] = np.nan

df_events.info()
df_events.head()

In [ ]:
i = 0
col_names = {}
for col in range(df_events.columns.start, df_events.columns.stop):
    col_names[col] = 'event_'+str(col + 1)
df_events.rename(columns=col_names, inplace=True)

df_events.head()

In [ ]:
df = df.drop("events", axis=1)
df.head()

In [ ]:
df = df.join(df_events)
df.head()

In [ ]:
df.to_csv(path + "metrics_events_split.csv")